In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Note - this is for pytorch versions before 0.4
# This is for accuracy in top 3 - you can change the first line to be whatever accuracy you want
# def accuracy_topk(output, target, topk=(3,)):
#     """Computes the precision@k for the specified values of k"""
#     maxk = max(topk)
#     batch_size = target.size(0)

#     _, pred = output.topk(maxk, 1, True, True)
#     pred = pred.t()
#     correct = pred.eq(target.view(1, -1).expand_as(pred))

#     res = []
#     for k in topk:
#         correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
#         res.append(correct_k.mul_(100.0 / batch_size))
#     return res

Using TensorFlow backend.


/kaggle/input/samplesub/Sample_Submissionb095668.csv
/kaggle/input/amazon-hackerearth/test.csv
/kaggle/input/amazon-hackerearth/train.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/amazon-hackerearth/train.csv')
test_df = pd.read_csv('/kaggle/input/amazon-hackerearth/test.csv')
submission = pd.read_csv('/kaggle/input/samplesub/Sample_Submissionb095668.csv')

In [3]:
display(train_df.head())
display(test_df.head())

,Review Text,Review Title,topic
0,"Did nothing for me, didn't help lost even with...",Useless,Shipment and delivery
1,"Did nothing for me, didn't help lost even with...",Useless,Not Effective
2,I have bought these bags and immediately open...,TRASH!!! Do not buy these bags it’s a waist of...,Customer Service
3,Gave me an allergic reaction on my face :(,Do not recommend,Allergic
4,These don't compare to the name brand wipes. F...,Can't tackle big messes,Texture


,Review Text,Review Title
0,I use chia seed in my protein shakes. These ta...,Bad tast
1,I use chia seed in my protein shakes. These ta...,Bad tast
2,Don’t waste your money.,No change. No results.
3,I use the book 'Fortify Your Life' by Tieraona...,"Good Vegan Choice, Poor Non Vegan Choice"
4,I use the book 'Fortify Your Life' by Tieraona...,"Good Vegan Choice, Poor Non Vegan Choice"


In [4]:
display(train_df['topic'].unique())

array(['Shipment and delivery', 'Not Effective', 'Customer Service',
       'Allergic', 'Texture', 'Quality/Contaminated', 'Color and texture',
       'Bad Taste/Flavor', 'Too big to swallow', 'Smells Bad',
       'Too Sweet', 'Ingredients', 'Expiry', 'Packaging',
       'Wrong Product received', 'Pricing', 'False Advertisement',
       'Inferior to competitors', "Didn't Like", 'Customer Issues',
       'Hard to Chew'], dtype=object)

In [5]:
# train_df['rtext_len'] = train_df['Review Text'].apply(len)
# train_df['rtitle_len'] = train_df['Review Title'].apply(len)
# display(train_df.groupby('rtext_len').describe().sample(10))
# display(train_df.groupby('rtitle_len').describe().sample(10))


    Word Count - Total number of words in the headline
    Character Count - Total number of characters in the headline excluding spaces
    Word Density - Average length of the words used in the headline
    Punctuation Count - Total number of punctuations used in the headline
    Upper-Case to Lower-Case Words ratio - ratio of upper case words used and lower case words used in the text

    Sentiment: Polarity - sentiment value of the headline computed using textblob package
    Part of Speech: Nouns to Verbs Ratio - ratio of nouns and verbs used in the text

In [6]:
# from sklearn.feature_extraction.text import CountVectorizer


# cv = CountVectorizer(stop_words = 'english')
# words = cv.fit_transform(data.verified_reviews)
# sum_words = words.sum(axis=0)


# words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
# words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
# frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])

# plt.style.use('fivethirtyeight')
# color = plt.cm.ocean(np.linspace(0, 1, 20))
# frequency.head(20).plot(x='word', y='freq', kind='bar', figsize=(15, 6), color = color)
# plt.title("Most Frequently Occuring Words - Top 20")
# plt.show()

In [7]:
# from wordcloud import WordCloud

# wordcloud = WordCloud(background_color = 'lightcyan', width = 2000, height = 2000).generate_from_frequencies(dict(words_freq))

# plt.style.use('fivethirtyeight')
# plt.figure(figsize=(10, 10))
# plt.axis('off')
# plt.imshow(wordcloud)
# plt.title("Vocabulary from Reviews", fontsize = 20)
# plt.show()

In [8]:
# df['word_count'] = df['Reviews Text'].apply(lambda x : len(x.split()))
# df['char_count'] = df['headline_text'].apply(lambda x : len(x.replace(" ","")))
# df['word_density'] = df['word_count'] / (df['char_count'] + 1)
# df['punc_count'] = df['headline_text'].apply(lambda x : len([a for a in x if a in punc]))

# df[['word_count', 'char_count', 'word_density', 'punc_count']].head(10)

In [9]:
train_df['text_title'] = train_df['Review Text'] + ' ' + train_df['Review Title']

In [10]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.topic.values)

In [11]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.text_title.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [12]:
print ('train shape ', xtrain.shape)
print ('test shape', xvalid.shape)

train shape  (4767,)
test shape (1192,)


In [13]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=2,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [14]:
# #both preds and truths are same shape m by n (m is number of predictions and n is number of classes)
# def top_n_accuracy(preds, truths, n):
#     best_n = np.argsort(preds, axis=1)[:,-n:]
#     ts = np.argmax(truths, axis=1)
#     successes = 0
#     for i in range(ts.shape[0]):
#       if ts[i] in best_n[i,:]:
#         successes += 1
#     return float(successes)/ts.shape[0]

In [15]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)

print(metrics.accuracy_score(yvalid, predictions))
# print(top_n_accuracy(predictions, yvalid, n=1))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.4714765100671141


In [16]:
test_df['text_title'] = test_df['Review Text'] + ' ' + test_df['Review Title']
test_tfv = tfv.transform(test_df['text_title'].values)
test_pred = clf.predict(test_tfv)
test_df['topic'] = lbl_enc.inverse_transform(test_pred)

In [17]:
final = test_df[["Review Text", "Review Title", "topic"]].copy()
final.to_csv('submission.csv', index=False)